In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
print(tf.__version__)
import nibabel as nib
import numpy as np
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv3D, MaxPooling3D, Conv3DTranspose
from tensorflow.keras.optimizers import RMSprop, Adam
import skimage
from skimage.util import montage
from skimage.transform import resize
#from skimage.util.montage import montage2d
from google.colab.patches import cv2_imshow
import cv2

In [ ]:
sample_filename="/content/drive/MyDrive/Kaggle/Brain_MRI/Data/MICCAI_BraTS2020_TrainingData/BraTS20_Training_082/BraTS20_Training_082_t2.nii"
sample_img = nib.load(sample_filename)
sample_img = np.asanyarray(sample_img.dataobj)
print(sample_img.shape)
img_min = np.min(sample_img)
sample_img=(sample_img - img_min) / (np.max(sample_img) - img_min)
print(img_min,np.max(sample_img))
cv2_imshow(sample_img[sample_img.shape[0]//2])
a=montage(sample_img)
print(a.shape)
cv2_imshow(montage(sample_img)*255)

sample_filename="/content/drive/MyDrive/Kaggle/Brain_MRI/Data/MICCAI_BraTS2020_TrainingData/BraTS20_Training_082/BraTS20_Training_082_seg.nii"
sample_img = nib.load(sample_filename)
sample_img = np.asanyarray(sample_img.dataobj)
print(sample_img.shape)
sample_img=resize(sample_img, (120, 120,80), preserve_range=True)
print(sample_img.shape)
img_min = np.min(sample_img)
sample_img=(sample_img - img_min)
sample_img[sample_img>0]=1
print(np.min(sample_img),np.max(sample_img))
cv2_imshow(sample_img[sample_img.shape[0]//2])
a=montage(sample_img)
print(a.shape)
cv2_imshow(montage(sample_img)*255)

In [ ]:
data_root="/content/drive/MyDrive/Kaggle/Brain_MRI/Data/MICCAI_BraTS2020_TrainingData"
images = os.listdir(data_root)
X_train=[]
Y_train=[]
#print(images)
for image in images:#[:10]:
  im=nib.load(data_root+"/"+image+"/"+image+"_t2.nii")
  im = np.array(im.dataobj)
  im=resize(im, (120, 120,80), preserve_range=True)
  im_min = np.min(im)
  im_max = np.max(im)
  #print(im_min,im_max)
  im=((im - im_min)*255) / (im_max - im_min)
  im=im.reshape(120,120,80,1)
  X_train.append(im)

  im=nib.load(data_root+"/"+image+"/"+image+"_seg.nii")
  im = np.asanyarray(im.dataobj)
  im=resize(im, (120, 120,80), preserve_range=True)
  im_min = np.min(im)
  im_max = np.max(im)
  im=(im - im_min) / (im_max - im_min)
  im[im>0]=1
  im=im.reshape(120,120,80,1)
  Y_train.append(im)

X_train=np.array(X_train)
Y_train=np.array(Y_train)
print(X_train.shape)
print(Y_train.shape)

In [ ]:
def get_model():
  inputs = Input((120,120,80,1))
  conv1 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(inputs)
  conv1 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(conv1)
  #pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
  pool1 = Conv3D(8, (3, 3, 3),(2, 2, 2), activation='relu', padding='same')(conv1)

  conv2 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(pool1)
  conv2 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv2)
  #pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
  pool2 = Conv3D(16, (3, 3, 3),(2, 2, 2), activation='relu', padding='same')(conv2)

  conv3 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(pool2)
  conv3 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv3)
  #pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
  pool3 = Conv3D(32, (3, 3, 3),(2, 2, 2), activation='relu', padding='same')(conv3)

  conv4 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(pool3)
  conv4 = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(conv4)
  #pool4 = MaxPooling3D(pool_size=(2, 2, 2))(conv4)

  #conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(pool4)
  #conv5 = Conv3D(512, (3, 3, 3), activation='relu', padding='same')(conv5)

  #conv5_1=Conv3DTranspose(256, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv5)

  #up6 = concatenate([conv5_1, conv4], axis=4)
  #conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(up6)
  #conv6 = Conv3D(256, (3, 3, 3), activation='relu', padding='same')(conv6)

  up7 = concatenate([Conv3DTranspose(32, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv4), conv3], axis=4)
  conv7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(up7)
  conv7 = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(conv7)

  up8 = concatenate([Conv3DTranspose(16, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv7), conv2], axis=4)
  conv8 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(up8)
  conv8 = Conv3D(16, (3, 3, 3), activation='relu', padding='same')(conv8)

  up9 = concatenate([Conv3DTranspose(8, (2, 2, 2), strides=(2, 2, 2), padding='same')(conv8), conv1], axis=4)
  conv9 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(up9)
  conv9 = Conv3D(8, (3, 3, 3), activation='relu', padding='same')(conv9)

  #conv10 = Conv3D(1, (1, 1, 1), activation='softsign')(conv9)
  conv10 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv9)
  #conv10 = Conv3D(1, (1, 1, 1), activation='sigmoid')(conv10)

  model = Model(inputs=[inputs], outputs=[conv10])
  model.summary()
  model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
model=get_model()

In [ ]:
def train_model(data,mask,n_epochs,batch_size,weights,checkpoint_dir):
 model = get_model()
 if weights!=0:
   #Loading previous trained models
		model.load_weights(checkpoint_dir+'/mask_'+str(weights)+'.h5')
 n_batches=int(data.shape[0]/batch_size)
 #print(n_batches)

 for epoch in range(n_epochs):
   print("Epoch : {}/{}".format(epoch+1+weights, n_epochs+weights))

   for i in range(n_batches):
     model.trainable = True
     loss = model.train_on_batch(data[i*batch_size:(i+1)*batch_size],mask[i*batch_size:(i+1)*batch_size])
     model.trainable = False
     print("Model Loss(Batch: {}): {}".format(i,loss))
     #generate_and_save_images(generator, epoch)
   if (epoch+1)%10==0:
     generate_and_save_images(model,X_train[1:2])
     #display and save output image for every 100 epoch
     model.save_weights(checkpoint_dir+'/mask_'+str(epoch+1+weights)+'.h5',True)

In [ ]:
def generate_and_save_images(model,input):
  #input=input.reshape((1,120,120,80,1))
  #print(input.shape)
  output=(model.predict(input))
  print(output.shape)
  input=input.reshape(120,120,80)
  output=output.reshape(120,120,80)
  output=montage(np.rot90(output))*255
  input1=(montage(np.rot90(input)))#*255)
  input1=input1.copy().astype('uint8') 
  i=10
  output[output>=i]=255
  output[output<i]=0
  #print(np.max(input))
  input1=cv2.cvtColor(input1, cv2.COLOR_GRAY2RGB)
  cv2.imwrite("/content/drive/MyDrive/Kaggle/Brain_MRI/outputs/img1.jpg",input1)
  #cv2_imshow(input1)
  input1[output>=128]=(1, 190, 200)
  #cv2_imshow(output)
  cv2_imshow(input1)
  cv2.imwrite("/content/drive/MyDrive/Kaggle/Brain_MRI/outputs/img.jpg",input1)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 10
checkpoint_dir="/content/drive/MyDrive/Kaggle/Brain_MRI/training_checkpoints"
Continue_training_step_number=0
train_model(X_train,Y_train,n_epochs=EPOCHS,batch_size=BATCH_SIZE,weights=Continue_training_step_number,checkpoint_dir=checkpoint_dir)

In [ ]:
weights=0
model = get_model()
model.load_weights(checkpoint_dir+'/mask_'+str(weights)+'.h5')
generate_and_save_images(model,X_train[4:5])